# Tensorflow: Basic Text Classification - Multiclass Label, with Hyperparameter Tuning via HParams

Reference:
* https://www.tensorflow.org/tutorials/keras/text_classification#exercise_multi-class_classification_on_stack_overflow_questions
* https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

## Setup

In [4]:
import io, os, re, shutil
import string
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 300

import tensorflow as tf
from tensorflow.keras import layers, losses

# Hyperparameter tuner using Keras Tuner
import keras_tuner as kt

%load_ext tensorboard

## Download the Stack Overflow Dataset

In [5]:
# Prepare to download the dataset to be used to the location CACHE_DIR/DATASET_SUBDIR
CACHE_DIR = '.\\data'
DATASET_URL = "http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
DATASET_SUBDIR = 'stackoverflowdb'
DATASET_ZIPFILE = 'stack_overflow_16k.tar.gz'

In [6]:
# Download the targeted dataset
# dataset = tf.keras.utils.get_file(
#             DATASET_ZIPFILE, DATASET_URL,
#             untar=True, cache_dir=CACHE_DIR, cache_subdir=DATASET_SUBDIR)

In [7]:
dataset_dir = os.path.join( CACHE_DIR, DATASET_SUBDIR )
print(dataset_dir)
os.listdir(dataset_dir)

.\data\stackoverflowdb


['README.md', 'stack_overflow_16k.tar.gz', 'test', 'train']

In [8]:
# Get the directory name of the training data
train_dir = os.path.join(dataset_dir, 'train')
print(train_dir)
os.listdir(train_dir)

.\data\stackoverflowdb\train


['csharp', 'java', 'javascript', 'python']

In [9]:
test_dir = os.path.join(dataset_dir, 'test')
print(test_dir)
os.listdir(test_dir)

.\data\stackoverflowdb\test


['csharp', 'java', 'javascript', 'python']

In [10]:
# # Remove directory containing unsupervised examples
# remove_dir = os.path.join(train_dir, 'unsup')
# shutil.rmtree(remove_dir)

In [11]:
batch_size = 32
seed = 42
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
            train_dir, batch_size=batch_size,
            validation_split=0.2, subset='training',
            seed=seed)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
            train_dir, batch_size=batch_size,
            validation_split=0.2, subset='validation',
            seed=seed)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
            test_dir, batch_size=batch_size)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 8000 files belonging to 4 classes.


In [12]:
for i,c in enumerate(raw_train_ds.class_names):
    print(f"Label: {i} => Class: {c}")

Label: 0 => Class: csharp
Label: 1 => Class: java
Label: 2 => Class: javascript
Label: 3 => Class: python


In [13]:
# for text_batch, label_batch in raw_train_ds.take(1):
#   for i in range(1):
#     print(f"Label: {label_batch.numpy()[i]} ({raw_train_ds.class_names[label_batch.numpy()[i]]})"
#           f" => Post: {text_batch.numpy()[i]}")

## Text Preprocessing

In [14]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    stripped_punct = tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')
    return stripped_punct

In [15]:
# Vocabulary size (features) and number of words in a sequence (review).
# max_features = 10000
# sequence_length = 250
max_features = 15000
sequence_length = 350

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = layers.TextVectorization(
                    standardize=custom_standardization,
                    max_tokens=max_features,
                    output_mode='int',
                    output_sequence_length=sequence_length)

In [16]:
# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [17]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [18]:
# # retrieve a batch (of 32 reviews and labels) from the dataset
# text_batch, label_batch = next(iter(raw_train_ds))
# first_review, first_label = text_batch[0], label_batch[0]
# print("Review", first_review)
# print("Label", raw_train_ds.class_names[first_label])
# print("Vectorized review", vectorize_text(first_review, first_label))

In [19]:
# print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
# print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
# print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

In [20]:
# Create vectorized training, validation, and test datasets
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [21]:
# [ x for x in train_ds.take(1).as_numpy_iterator() ]
# print(dir(train_ds))

## Configure the Dataset for Performance

In [22]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Create a function to build and compile a model

In [23]:
def model_builder(hp):
    model = tf.keras.Sequential()
        
    # Tune the embedding dimensions
    hp_embedding_dims = hp.Int('embedding_dimensions', min_value=16, max_value=64, step=16)
    model.add( layers.Embedding( max_features+1, hp_embedding_dims ) )

    # Tune the dropout rate in the first Dropout layer
    hp_dropout_rate_1 = hp.Float('dropout_rate_1', min_value=0.0, max_value=0.3, step=0.1)
    model.add( layers.Dropout(hp_dropout_rate_1) )
    
    model.add( layers.GlobalAveragePooling1D() )

    # Tune the dropout rate in the second Dropout layer
    hp_dropout_rate_2 = hp.Float('dropout_rate_2', min_value=0.0, max_value=0.3, step=0.1)
    model.add( layers.Dropout(hp_dropout_rate_2) )
    
    model.add( layers.Dense(4) )
    
    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])    
    model.compile(
        optimizer=tf.keras.optimizers.Adam( learning_rate=hp_learning_rate ),
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
        )    
    
    return model


## Create a Classification Model

In [24]:
# embedding_dim = 32
# model = tf.keras.Sequential( [
#     layers.Embedding( max_features+1, embedding_dim ),
#     layers.Dropout(0.2),
#     layers.GlobalAveragePooling1D(),
#     layers.Dropout(0.2),
#     layers.Dense(4)
#     ])

# model.summary()

## Compile and Train the Model

In [25]:
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')
LOGS_ROOTDIR = os.path.join('.', 'logs')
this_run_log_dir = os.path.join( LOGS_ROOTDIR, "fit_" + dt.datetime.now().strftime("%Y%m%d-%H%M%S") )
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=this_run_log_dir, histogram_freq=1)
print(f"Prepared Tensorboard log dir: {this_run_log_dir}")

# Stop epochs if validation loss is increasing
earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Prepared Tensorboard log dir: .\logs\fit_20211119-153512


In [26]:
# model.compile(
#     optimizer='adam',
#     loss=losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy']
#     )

In [27]:
# epochs = 100
# history = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=epochs,
#     callbacks=[tensorboard_callback, earlystopping_callback]
#     )

## Instantiate the Tuner and Perform Hypertuning

In [28]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=3,
                     directory='hpt_dir',
                     project_name='nlp_multiclass_hpt')

In [ ]:
tuner.search(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[earlystopping_callback]
    )

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters( num_trials=1 )[0]
print(f"Optimal hyperparameters:\n{best_hps}")

Trial 53 Complete [00h 00m 09s]
val_accuracy: 0.29124999046325684

Best val_accuracy So Far: 0.7943750023841858
Total elapsed time: 00h 07m 10s

Search: Running Trial #54

Hyperparameter    |Value             |Best Value So Far 
embedding_dimen...|32                |48                
dropout_rate_1    |0.2               |0.2               
dropout_rate_2    |0.2               |0.2               
learning_rate     |0.001             |0.01              
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |0                 |0                 

Epoch 1/2


## Train the Tuned Model

In [ ]:
# Stop epochs if validation loss is increasing
earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Train the model with the optimal hyperparameters obtained from the tuner search above
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[tensorboard_callback, earlystopping_callback]
    )

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
# Reinstantiate and retrain the model with the
# optimal hyperparameters and optimal number of epochs determined above
hypermodel = tuner.hypermodel.build(best_hps)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=best_epoch,
    callbacks=[tensorboard_callback]
    )

## Evaluate the Model

In [ ]:
loss, accuracy = hypermodel.evaluate(test_ds)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

history_dict = history.history
print( history_dict.keys() )

plt.plot( 'accuracy', 'b-', data=history.history )
plt.plot( 'val_accuracy', 'g:', data=history.history )
# plt.plot( [accuracy]*len(history.history['binary_accuracy']), 'r--', data=history.history )
y_axis_min_val = max( 0, round(min(accuracy, min(history.history['val_accuracy']), min(history.history['accuracy']) )-0.1, 1) )
_ = plt.ylim( y_axis_min_val, 1.0)
plt.axhline( accuracy, color='r', linestyle=':')

In [ ]:
%tensorboard --logdir logs

## Export the Model

In [ ]:
# Wrap the trained model to permit it to be used directly in input strings (vs. vectors)
# and then apply an activation layer to yield a floating result between 0 and 1
export_model = tf.keras.Sequential([
    vectorize_layer,
    hypermodel,
    layers.Softmax()
    ])

export_model.summary()

In [ ]:
export_model.compile(
    optimizer="adam",
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
    )

In [ ]:
# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print(accuracy)

In [ ]:
# Some test examples
n_examples = 16
examples_np = [ x for x in raw_test_ds.take(n_examples).as_numpy_iterator() ]
examples = [ x[0][0].decode() for x in examples_np ]
true_labels = [ x[1][0] for x in examples_np ]
true_classes = [ raw_train_ds.class_names[label] for label in true_labels ]

preds = export_model.predict(examples)
pred_labels = list(np.argmax(preds, axis=1))
pred_classes = [ raw_train_ds.class_names[label] for label in pred_labels ]
r_df = pd.concat([
        pd.DataFrame( {'Post': examples,
                       'True Label': true_labels, 'True Class': true_classes,
                       'Predicted Label': pred_labels, 'Predicted Class': pred_classes
                      }),
        pd.DataFrame( preds, columns=['c0 (csharp)', 'c1 (java)', 'c2 (javascript)', 'c3 (python)'] ),
        pd.DataFrame( np.max(preds, axis=1) / np.mean(preds, axis=1), columns=['max/mean'] )
        ], axis=1 )
r_df['Prediction Correct'] = r_df['Predicted Label'] == r_df['True Label']
r_df = r_df[['Post', 'True Label', 'True Class', 'Predicted Label',
       'Predicted Class', 'c0 (csharp)', 'c1 (java)', 'c2 (javascript)',
       'c3 (python)', 'max/mean', 'Prediction Correct']]
r_df.sort_values(by=['Prediction Correct', 'max/mean'], ascending=[False, False], inplace=True)
r_df

## Retrieve the Trained Word Embeddings and Save them to Disk

In [ ]:
# weights = model.get_layer('embedding').get_weights()[0]
# vocab = vectorize_layer.get_vocabulary()

# with io.open('./data/vectors.tsv', 'w', encoding='utf-8') as out_v:
#     with io.open('./data/metadata.tsv', 'w', encoding='utf-8') as out_m:
#         for index, word in enumerate(vocab):
#             if index == 0:
#                 continue    # skip 0, it's padding
                
#             vec = weights[index]
#             out_v.write( '\t'.join( [ str(x) for x in vec ]) + '\n' )
#             out_m.write( word + '\n' )


## Visualize the Embeddings
* Tensorflow Embedding Projector: http://projector.tensorflow.org/
* Click on "Load data".
* Upload the two files you created above: vecs.tsv and meta.tsv.